In [2]:
import pandas as pd
import numpy as np
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

ModuleNotFoundError: No module named 'fosforml'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt 
plt.style.use('fivethirtyeight')

In [ ]:
my_session.connection.database

In [ ]:
'ASSORTMENT_PLANNING'

In [ ]:
my_session.connection.schema

In [ ]:
'CPG_BRONZE'

In [ ]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [ ]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [ ]:
type(sf_df)

In [ ]:
snowflake.snowpark.dataframe.DataFrame

In [ ]:
df=sf_df.to_pandas()

In [ ]:
pip install --upgrade fosforml

     |████████████████████████████████| 51kB 3.6MB/s eta 0:00:011
     |████████████████████████████████| 1.9MB 14.1MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 97.7MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 76.6MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 125kB/s s eta 0:00:01B/s eta 0:00:02
     |████████████████████████████████| 133kB 76.9MB/s eta 0:00:01
     |████████████████████████████████| 174kB 100.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 16.9MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 93.4MB/s eta 0:00:01
     |████████████████████████████████| 98.7MB 264kB/s  eta 0:00:01
     |████████████████████████████████| 1.3MB 86.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 14.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 6.1MB/s  eta 0:00:01
     |████████████████████████████████| 2.5MB 89.6MB/s eta 0:00:01
     |████████████████████████████████| 307